In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.linalg import block_diag

plt.close('all')

Loading the data

In [22]:
data = np.load('data_point_land_1.npz', allow_pickle=True)

#print(data.files)

Meas = data['Meas'] #Landmark measurements
Uf = data['Uf'] #measured forward velocity (odometry)
Ua = data['Ua'] #measured angular velocity (odometry)
Q = data['Q']
Qturn = data['Qturn']
R = data['R']
Nland = data['Nland'] #number of Landmarks
Ts = data['Ts']
Wturn = data['wturn'] #treshold Wturn
Pose = data['Pose'] #data to be used only for comparison (x(t), y(t), theta(t) of the robot)
Landmarks = data['Landmarks'] #data to be used only for comparison (ith row corresponds to i+1th of landmark locations.)

In [23]:
range_land = Meas.item()['range'] #landmark ranges
angle_land = Meas.item()['angle']  #landmark angles
index_land = Meas.item()['land']  #landmark indices

Angle Wrapping Function

In [176]:
def angle_wrap(angle):
    angle = (angle + np.pi) % (2 * np.pi) - np.pi
    return angle

def innovation(M, xp):
    x = xp[0]
    y = xp[1]
    theta = xp[2]
    landmarks = M[:, 2]

    # take the predictions
    predicted_meas = np.zeros((len(landmarks), 2))
    
    for i in range(len(landmarks)):
        j = int(landmarks[i] - 1)

        lx = xp[3 + 2*j]
        ly = xp[3 + 2*j + 1]
        
        predicted_p = np.sqrt((lx - x)**2 + (ly - y)**2)
        predicted_alpha = angle_wrap(np.arctan2(ly - y, lx - x)) - theta
        predicted_meas[i][0] = predicted_p
        predicted_meas[i][1] = predicted_alpha
    
    inn = M[:, :2] - predicted_meas
    return inn.reshape(-1,1)

Initialising

In [177]:
N = Uf.shape[0] #Number of odometry measurements

n_upper = 3 #upper system order: x,y,theta
n_lower = Nland.item()*2 #lower system order: 2 for every landmark (x,y)
n = n_upper + n_lower  #system order

x0 = np.zeros(n) #initial states
x0[0] = Pose[0,0] #x(0)
x0[1] = Pose[0,1] #y(0)
x0[2] = Pose[0,2] #theta(0)
#upper covariance
lambda_ = 0.01
P_upper = lambda_ * np.eye(n_upper) #uncertainty of x,y,theta
upper_zeros = np.zeros((n_upper,n_lower))
#lower covariance
eta = 1000
P_lower = eta * np.eye(n_lower) #uncertainty landmarks
lower_zeros = np.zeros((n_lower,n_upper))
#Initial covariance
P0 = np.block([[P_upper,upper_zeros],
              [lower_zeros,P_lower]])
Xp = x0.copy()
Pp = P0.copy()

EKF loop

In [175]:
X_pred = np.empty((N,n))
P_pred = np.empty((N,n))
i = 0

checked_landmarks = []
while i < N:
    #prediction
    Xp_lower = np.zeros(n_lower)
    # isn't it: np.concatenate((Ts*np.array([Uf[i]*np.cos(Xp[2]), Uf[i]*np.sin(Xp[2]), Ua[i]]), Xp_lower))
    # Xp = Xp + np.concatenate((np.array([Uf[i]*np.cos(Xp[2]), Uf[i]*np.cos(Xp[2]), Ua[i]]), Xp_lower))
    Xp = Xp + np.concatenate((Ts*np.array([Uf[i]*np.cos(Xp[2]), Uf[i]*np.sin(Xp[2]), Ua[i]]), Xp_lower))
    dUpper_dx = np.array([[1, 0, -Ts*Uf[i]*np.sin(Xp[2])],
                          [0, 1, Ts*Uf[i]*np.cos(Xp[2])],
                          [0, 0, 1]])
    dLower_dx = np.eye(n_lower)
    F = np.block([[dUpper_dx,upper_zeros],
                [lower_zeros,dLower_dx]])
    # print(F)
    dupper_dw = np.array([[-Ts*np.cos(Xp[2]), 0],
                  [-Ts*np.sin(Xp[2]), 0],
                  [0, -Ts]])
    dlower_dw = np.zeros((n_lower, 2))
    G = np.block([[dupper_dw],
                  [dlower_dw]])
    
    if Ua[i] >= Wturn: #If turn rate is bigger than a given turn rate:
        Q = Qturn

    Pp = F@Pp@F.T + G@Q@G.T


    #correction (I dont know if correction should be before or after prediction. Maybe first a correction during initialization, then prediction then another correction.)
    
    """
        ########### Correction ###########
        
        Find the landmark and correct
        We need two Jacobians
    """
    # build H
    x = Xp[0]
    y = Xp[1]
    theta = Xp[2]
    H = np.zeros((len(range_land[i])*2, n))
    for k in range(len(range_land[i])):
        # landmarks are from 1, so when we see landmark 1 it means that it will
        # be on the 0 position after x, y, theta - Lx = 3, Ly = 4
        j = index_land[i][k] - 1 
        
        
        lx = Xp[3 + 2*j]
        ly = Xp[3 + 2*j + 1]
        
        row_p = k*2
        row_alpha = k*2 + 1
        # derivatives with respect to m_p
        p_den = np.sqrt((lx - x)**2 + (ly - y)**2) 
        
        H[row_p][0] = (x - lx)/p_den # dx
        H[row_p][1] = (y - ly)/p_den # dy
        H[row_p][2] = 0 # dtheta
        H[row_p][3 + 2*j] = (lx - x)/p_den #dlx
        H[row_p][3 + 2*j+1] = (ly - y)/p_den #dly
        
        
        # derivatives with respect to m_alpha
        alpha_den = (lx - x)**2 + (ly - y)**2
        H[row_alpha][0] = (ly - y)/alpha_den # dx
        H[row_alpha][1] = -(lx - x)/alpha_den #dy
        H[row_alpha][2] = -1 #dtheta
        H[row_alpha][3 + 2*j] = -(ly - y)/alpha_den #dlx
        H[row_alpha][3 + 2*j + 1] = (lx - x)/alpha_den #dly
    
    # build R
    R_new = np.kron(np.eye(len(range_land[i])), R)
    # Equations
    K = Pp @ H.T @ np.linalg.inv(H @ Pp @ H.T + R_new)
    P = Pp - K @ H @ Pp
    measurement =  np.array([range_land[i], angle_land[i], index_land[i]]).T
    inn = innovation(measurement, Xp)
    X = Xp - K @ inn
    
    
    
    break
    i += 1




(6, 1)
(39, 6)
[[40.92856218  5.82975292  0.79451779 ...  0.57604825  0.57604825
   0.57604825]
 [42.00490114  6.90609188  1.87085674 ...  1.65238721  1.65238721
   1.65238721]
 [40.40034712  5.30153787  0.26630273 ...  0.0478332   0.0478332
   0.0478332 ]
 ...
 [40.35251393  5.25370467  0.21846953 ...  0.          0.
   0.        ]
 [40.35251393  5.25370467  0.21846953 ...  0.          0.
   0.        ]
 [40.35251393  5.25370467  0.21846953 ...  0.          0.
   0.        ]]


In [ ]:
#before correction step: dont forget to insert the landmark into state vector the first time. After inserting don't insert the same one again.
# if landmark
#     L_x[i] = blaba
#     L_y[i] = blabla